# Netscience Collaboration Network

data source: http://www-personal.umich.edu/~mejn/netdata/

citation:
```
@article{newman2006finding,
     title={Finding community structure in networks using the eigenvectors of matrices},
     author={Newman, Mark EJ},
     journal={Physical review E},
     volume={74},
     number={3},
     pages={036104},
     year={2006},
     publisher={APS}
}
```

In [2]:
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt

### Construct graph from network data

In [3]:
G_all = nx.read_gml("netscience.gml")
nx.info(G_all)

'Graph with 1589 nodes and 2742 edges'

In [4]:
LCC = max(nx.connected_components(G_all), key=len)
G = G_all.subgraph(LCC).copy()
nx.info(G)

'Graph with 379 nodes and 914 edges'

In [5]:
pos = nx.spring_layout(G, dim=3)

In [6]:
layout = Layout(G, pos)

NameError: name 'Layout' is not defined

In [7]:
layout.node_names = list(G.nodes)

NameError: name 'layout' is not defined

In [8]:
weights = [G[n1][n2]["value"] for n1, n2 in G.edges]
min_weight = np.min(weights)
max_weight = np.max(weights)

max_glow = 255
min_glow = 100

def get_glow(weight):
    k = (max_glow - min_glow) / (max_weight - min_weight)
    d = min_glow - k * min_weight
    return int(k * weight + d)

layout.edge_colors = [(255, 255, 255, get_glow(G[n1][n2]["value"])) for n1, n2 in G.edges]

NameError: name 'layout' is not defined

In [9]:
layout.node_table

NameError: name 'layout' is not defined

In [10]:
layout.edge_table

NameError: name 'layout' is not defined

In [11]:
layout.write(node_file_path="collaboration_nodes.csv", edge_file_path="collaboration_edges.csv")

NameError: name 'layout' is not defined

### Persistent Homology

https://snap.stanford.edu/social2012/papers/carstens-horadam.pdf

In [ ]:
filtration = []
thresholds = np.linspace(min_weight, max_weight, len(G))
for threshold in thresholds:
    filter_edge = lambda n1, n2: G[n1][n2]["value"] >= threshold
    view = nx.subgraph_view(G, filter_edge=filter_edge).copy()
    filtration.append(view)

betti_0 = [nx.number_connected_components(view) for view in filtration]
w_star = thresholds[::-1][np.argmin(betti_0[::-1])]
n_edges = [view.number_of_edges() for view in filtration]

In [ ]:
plt.loglog(thresholds, betti_0, label="# connected components")
plt.loglog(thresholds, n_edges, label="# edges")
plt.vlines(w_star, 0, n_edges[0], color="black", alpha=0.5, label=fr"$w^*$ = {w_star:.3g}")
plt.xlabel("weight threshold")
plt.legend()
plt.show()